### **PINN Exercise!**

Solve the following problem with a PINN
$$
\begin{cases}
u'' = \cos(\pi x) & \text { in } (0,2),\\
u'(0) = 3 \\
u(2) = 0.  
\end{cases}
$$
Take inspiration from the exercises we have done in class.

In this case, the exact solution is
$$
u(x) = 3(x - 2) - \frac{1}{\pi^2}(\cos(\pi x) - 1).
$$

In [1]:
#### starting stuff ####

import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.input_layer = nn.Linear(1,5)
        self.hidden_layer1 = nn.Linear(5,5)
        self.hidden_layer2 = nn.Linear(5,5)
        self.hidden_layer3 = nn.Linear(5,5)
        self.hidden_layer4 = nn.Linear(5,5)
        self.output_layer = nn.Linear(5,1)

    def forward(self, x):
        input = x
        layer1_out = torch.sigmoid(self.input_layer(input))
        layer2_out = torch.sigmoid(self.hidden_layer1(layer1_out))
        layer3_out = torch.sigmoid(self.hidden_layer2(layer2_out))
        layer4_out = torch.sigmoid(self.hidden_layer3(layer3_out))
        layer5_out = torch.sigmoid(self.hidden_layer4(layer4_out))
        output = self.output_layer(layer5_out)
        return (x - 2)*output

In [3]:
### (2) Model
seed = 0
torch.manual_seed(seed)
net = Net()
mse_cost_function = torch.nn.MSELoss() # Mean squared error
optimizer = torch.optim.Adam(net.parameters())

In [4]:
## PDE as loss function. Thus would use the network which we call as u_theta
def R(x, net):
    u = net(x) # the dependent variable u is given by the network based on independent variables x,t
    
    ## Based on our R = du/dx - 2du/dt - u, we need du/dx and du/dt
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
    
    u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]
    
    f = torch.Tensor(np.cos(np.pi*x.detach().numpy()))
    pde = u_xx - f
    
    return pde

In [5]:
### Neumann Boundary

def Neumann(net):
    x_bc_n = np.zeros((1,1))
    x_bc_n[0] = 0;
    pt_x_bc = Variable(torch.from_numpy(x_bc_n).float(), requires_grad=True)
    u = net(pt_x_bc)
    u_x = torch.autograd.grad(u.sum(), pt_x_bc, create_graph=True)[0]
    
    neumann = u_x - 3.


    return neumann

In [6]:
## Data from Boundary Conditions
# u(x,0)=6e^(-3x)
## BC just gives us datapoints for training

# BC tells us that for any x in range[0,2] and time=0, the value of u is given by 6e^(-3x)
# Take say 500 random numbers of x
x_bc = np.zeros((1,1))
x_bc[0] = 2;
u_bc = np.zeros((1,1))


In [ ]:
### (3) Training / Fitting
iterations = 10000
for epoch in range(iterations):
    optimizer.zero_grad() # to make the gradients zero
    
    # Loss based on boundary conditions
    pt_x_bc = Variable(torch.from_numpy(x_bc).float(), requires_grad=False)
    pt_u_bc = Variable(torch.from_numpy(u_bc).float(), requires_grad=False)
    net_bc_out = net(pt_x_bc) # output of u(x)
    mse_u = mse_cost_function(net_bc_out, pt_u_bc)

    net_neumann = Neumann(net)
    zero_n = np.zeros((1,1))
    pt_zero_n = Variable(torch.from_numpy(zero_n).float(), requires_grad=False)
    mse_n = mse_cost_function(net_neumann, pt_zero_n)
    
    # Loss based on PDE
    x_collocation = np.random.uniform(low=0.0, high=2.0, size=(500,1))
    all_zeros = np.zeros((500,1))
    
    
    pt_x_collocation = Variable(torch.from_numpy(x_collocation).float(), requires_grad=True)
    pt_all_zeros = Variable(torch.from_numpy(all_zeros).float(), requires_grad=False)
    f_out = R(pt_x_collocation, net) # output of R(x)
    mse_f = mse_cost_function(f_out, pt_all_zeros)
    
    # Combining the loss functions
    loss =  mse_n + mse_f # + mse_u
    
    
    loss.backward() 
    optimizer.step()

    with torch.autograd.no_grad():
    	print(epoch,"Loss:",loss.item())

0 Loss: 9.990673065185547
1 Loss: 10.009625434875488
2 Loss: 9.955940246582031
3 Loss: 9.941473007202148
4 Loss: 9.921049118041992
5 Loss: 9.89205265045166
6 Loss: 9.854997634887695
7 Loss: 9.84577751159668
8 Loss: 9.81629467010498
9 Loss: 9.776972770690918
10 Loss: 9.793314933776855
11 Loss: 9.759598731994629
12 Loss: 9.756197929382324
13 Loss: 9.710345268249512
14 Loss: 9.654684066772461
15 Loss: 9.653103828430176
16 Loss: 9.642539024353027
17 Loss: 9.597256660461426
18 Loss: 9.548538208007812
19 Loss: 9.569509506225586
20 Loss: 9.555068969726562
21 Loss: 9.523658752441406
22 Loss: 9.504850387573242
23 Loss: 9.466285705566406
24 Loss: 9.462310791015625
25 Loss: 9.400184631347656
26 Loss: 9.401322364807129
27 Loss: 9.386116027832031
28 Loss: 9.346224784851074
29 Loss: 9.323200225830078
30 Loss: 9.282901763916016
31 Loss: 9.292330741882324
32 Loss: 9.265650749206543
33 Loss: 9.256840705871582
34 Loss: 9.230552673339844
35 Loss: 9.175607681274414
36 Loss: 9.162980079650879
37 Loss: 9.18

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

x=np.random.uniform(low=0.0, high=2.0, size=(100,1))

pt_x = Variable(torch.from_numpy(x).float(), requires_grad=True)
pt_u = net(pt_x)
u=pt_u.data.cpu().numpy()
plt.scatter(x, u)
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

x=np.random.uniform(low=0.0, high=2.0, size=(100,1))
print(np.cos(np.pi*2))
u=3*(x - 2) - (np.cos(np.pi*x) - 1)*(1/(np.pi**2))
plt.scatter(x, u)
plt.show()

To impose *directly* the boundary conditions (usually gives better training results) you have to: 
* impose the condition directly on the output of the forward law multiplying the output by a function that does the job!
In our case: to make the solution zero at $x = 2$, a good function is $x - 2$. Thus ``(x - 2)*output``.
* Get rid of the Dirichlet cost: it is not needed anymore :)